In [1]:
import numpy as np
import matplotlib.pyplot as plt
from test.util import TestSuite

In [2]:
def RMSE_Loss(actual, predicted):
    """Root Mean Squared Error"""
    return np.sqrt(np.mean(np.square(actual - predicted)))

In [3]:
def min_max_transform(series):
  series_temp = np.copy(series)
  for i in range(len(series_temp)):
    series_temp[i] = (series_temp[i] - np.max(series))/(np.max(series)-np.min(series)) + 1
  return series_temp

In [4]:
import os, shutil
DIR="experiment_linear_regression/"
FILE_NAME=DIR+"/log.csv"
if os.path.exists(DIR): shutil.rmtree(DIR)
if not os.path.exists(DIR): os.mkdir(DIR)

In [5]:
def outputExperiment(id, series_name, mixed, feature_length, target_length, target_offset, rmse_train, rmse_test):
    with open(FILE_NAME, 'a') as f:
        result = ", ".join(map(str, list([id, series_name , mixed , feature_length , target_length , target_offset , rmse_train , rmse_test])))
        print(result, file=f)

In [6]:
def showExperiment(ID, mixed, series, series_name, Y_train, Y_test, Yhat_train, Yhat_test, Yhat_series, rmse_train, rmse_test):
    plt.figure(figsize=(30,10))
    plt.subplot(311)
    plt.margins(0.05)
    plt.plot(series)
    plt.plot(Yhat_series)
    plt.legend(["True", "Predicted"])
    plt.title("Series: "+ series_name+" (Random Sampling="+str(mixed)+")")

    plt.subplot(323)
    plt.margins(0.05)
    plt.plot(Y_train)
    plt.plot(Yhat_train)
    plt.legend(["True", "Predicted"])
    plt.title("RMSE Train: "+ str(rmse_train))

    plt.subplot(324)
    plt.margins(0.05)
    plt.plot(Y_test)
    plt.plot(Yhat_test)
    plt.legend(["True", "Predicted"])
    plt.title("RMSE Test: "+ str(rmse_test))

    plt.subplot(325)
    plt.margins(0.05)
    plt.plot(abs(Y_train - Yhat_train))
    plt.legend(["Residuals"])
    plt.title("Train")

    plt.subplot(326)
    plt.margins(0.05)
    plt.plot(abs(Y_test - Yhat_test))
    plt.legend(["Residuals"])
    plt.title("Test")

    plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)
    plt.savefig(DIR+str(ID)+".png", transparent=True)

# Linear Regression

In [7]:
from src.models.regressor.linear_regression import Model_LinearRegression

In [8]:
def executeLinearRegression(X_train, X_test, Y_train, Y_test):
    model_linear_regression = Model_LinearRegression(error_fun=RMSE_Loss)
    yhat_train = model_linear_regression.__train__(X_train=X_train, Y_train=Y_train)
    rmse_train = model_linear_regression.__get_error_train__()

    yhat_test = model_linear_regression.__test__(X_test=X_test, Y_test=Y_test)
    rmse_test = model_linear_regression.__get_error_test__()

    yhat_series = model_linear_regression.__test__(X_test=np.concatenate((X_train, X_test), axis=0), Y_test=np.concatenate((Y_train, Y_test), axis=0))
    return rmse_train, rmse_test, yhat_train, yhat_test, yhat_series

In [9]:
def PipelineLinearRegression():
    synthetic_dataset_test_suite = TestSuite()

    synthetic_datasets =  synthetic_dataset_test_suite.__get_test_suite_size__()
    
    mixed = [True, False]
    feature_length = [1, 2, 3, 4, 5]
    target_offset = [1, 2, 3, 4, 5]
    target_length = [1]

    ID = 0
    print("\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format("ID", "Series", "Mixed", "Autoregressive Variables", "Target Length", "Target Offset", "RMSE Train", "RMSE Test"))
    for index in range(synthetic_datasets):
        series = synthetic_dataset_test_suite.__get_numpy_test_series_from_index__(index)
        for ind_mixed in mixed:
            for ind_feature in feature_length:
                for ind_offset in target_offset:
                    for ind_target in target_length:
                        X_train, X_test, Y_train, Y_test = synthetic_dataset_test_suite.__train_and_test_from_numpy_series__(
                            transform=min_max_transform,
                            series=series,
                            mixed=ind_mixed,
                            feature_length=ind_feature,
                            offset=ind_offset,
                            target_length=ind_target,
                            tensor = False
                        )
                        rmse_train, rmse_test, yhat_train, yhat_test, yhat_series = executeLinearRegression(X_train=X_train, X_test=X_test, Y_train=Y_train, Y_test=Y_test)
                        outputExperiment(
                            ID,
                            synthetic_dataset_test_suite.__get_name_test_series_from_index__(index),
                            ind_mixed,
                            ind_feature,
                            ind_offset,
                            ind_target,
                            rmse_train,
                            rmse_test
                        )
                        showExperiment(
                            ID,
                            ind_mixed,
                            np.concatenate((Y_train, Y_test), axis=0),
                            synthetic_dataset_test_suite.__get_name_test_series_from_index__(index),
                            Y_train,
                            Y_test,
                            yhat_train,
                            yhat_test,
                            yhat_series,
                            rmse_train,
                            rmse_test
                        )
                        ID += 1
    

In [10]:
PipelineLinearRegression()

	ID	Series	Mixed	Autoregressive Variables	Target Length	Target Offset	RMSE Train


AttributeError: 'TestSuite' object has no attribute '__train_and_test_from_numpy_series__'

In [ ]:
from cross_validation.regressor.linear_regression import KFoldCrossValidation_LinearRegression

In [ ]:
synthetic_dataset_test_suite = TestSuite()
cv_linear_regression = KFoldCrossValidation_LinearRegression(
    series = synthetic_dataset_test_suite.__get_numpy_test_series_from_index__(6),
    loss = RMSE_Loss,
    one_step_offset_target= 5
)
cross_validation_param = {
    "features_length": np.arange(1, 20, 1).tolist()
}
params, model = cv_linear_regression.search(cross_validation_param)

In [ ]:
params

In [ ]:
model

In [ ]:
_, X_test, _, Y_test = synthetic_dataset_test_suite.__train_and_test_from_numpy_series__(
    transform=min_max_transform,
    series=synthetic_dataset_test_suite.__get_numpy_test_series_from_index__(6),
    mixed=False,
    feature_length=params["features_length"],
    offset=1,
    target_length=1
)

In [ ]:
yhat = model.__test__(X_test, Y_test)

In [ ]:
plt.plot(Y_test)
plt.plot(yhat)